### Import Dependancies

In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from sqlalchemy import create_engine, MetaData, ForeignKey, Column, Integer, String, Date, Boolean, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy_utils import create_database, database_exists
from pg_config import pg_password, pg_user, pg_host, pg_port 

### Read in and clean data

In [172]:
# Read in all the csv files from the data folder and save them as a dictionary of DataFrames
data_dir = Path('data')
dfs = {}

try:
    for file_path in data_dir.glob('*.csv'):
        if file_path.is_file():
            # Get the filename without extension
            file_name = file_path.stem  
            df = pd.read_csv(file_path)
            dfs[file_name] = df
except Exception as e:
    print(f"Error occurred: {e}")

# Check the keys of the dictionary
print(dfs.keys())

# Save each DataFrame as individual dfs
for key, df in dfs.items():
    globals()[f'df_{key}'] = df

dict_keys(['photo_tags', 'photos', 'users', 'tags', 'comments', 'likes', 'follows'])


In [173]:
# Check for nulls in each df
for key, df in dfs.items():
    print(f"Nulls in DataFrame {key}:")
    print(df.isnull().sum().sum())


Nulls in DataFrame photo_tags:
0
Nulls in DataFrame photos:
0
Nulls in DataFrame users:
0
Nulls in DataFrame tags:
0
Nulls in DataFrame comments:
0
Nulls in DataFrame likes:
0
Nulls in DataFrame follows:
0


In [174]:
# Replace spaces with underscores in the column names and make all lowercase
for key, df in dfs.items():
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.lower()

In [175]:
# Change yes/no columns to boolean values and data type
for key, df in dfs.items():
    for col in df.columns:
        # If columns only contain yes or no, change to boolean
        if df[col].isin(['yes', 'no']).all():
            df[col] = df[col].map({'yes': True, 'no': False})

In [176]:
# Convert the created columns to datetime data type
date_time_cols = ['created_dat','created_date', 'created_time', 'created_timestamp']

for key, df in dfs.items():
    for col in date_time_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')  # Coerce will return NaT for invalid dates


In [177]:
# Create new id columns for the likes and follows dataframes as they will be used as foreign keys in the interactions table
df_likes['like_id'] = (df_likes.index + 1).astype(int)
df_follows['follow_id'] = df_follows.index + 1

In [178]:
# Rename the columns in each dataframe to match the database schema
df_photos.rename(columns={'created_dat': 'created_date','id':'photo_id'}, inplace=True)
df_users.rename(columns={'id': 'user_id','private/public':'private'}, inplace=True)
df_tags.rename(columns={'id': 'tag_id'}, inplace=True)
df_comments.rename(columns={'user__id': 'user_id','id':'comment_id','created_timestamp':'created_time'}, inplace=True)
df_likes.rename(columns={'user_': 'user_id', 'photo': 'photo_id'}, inplace=True)
df_follows.rename(columns={'follower': 'follower_user_id', 'followee_': 'user_id'}, inplace=True)

##### Create interaction dataframe from likes, comments, and follows dataframes

In [179]:
# Combine Likes, Comments, and Follows DataFrames into a single Interactions DataFrame
df_interactions = pd.concat([df_likes, df_comments, df_follows], ignore_index=True)

# Create interaction_id column
df_interactions['interaction_id'] = df_interactions.index + 1

# Add a new column to indicate the type of interaction ('like', 'comment', or 'follow')
df_interactions['interaction_type'] = pd.Series(['like'] * len(df_likes) + ['comment'] * len(df_comments) + ['follow'] * len(df_follows))

# Add an interaction date column (you can use 'created_time' column or any other appropriate date column)
df_interactions['interaction_date'] = df_interactions['created_time']

# List of columns to keep
keep_cols = ['interaction_id', 'interaction_type', 'interaction_date', 'user_id', 'photo_id', 'comment_id', 'like_id','follow_id']

# Remove columns that are not in the keep_cols list
df_interactions = df_interactions[[col for col in df_interactions.columns if col in keep_cols]]

# Make sure the id columns are integers even though they contain NaN values
id_cols = ['user_id', 'photo_id', 'comment_id', 'like_id', 'follow_id']
for col in id_cols:
    df_interactions[col] = df_interactions[col].astype('Int64')


### Create a PostgesSQL Database 
The database is designed in a star schema with a the central facts table being the Interactions table and all others the being the dimension tables. 

In [180]:
# Parameters for connecting to the database and creating the engine
db_params = {
    "host": "localhost",
    "port": int(pg_port),
    "database": "insta_db",
    "user": pg_user,
    "password": pg_password,
}

# Create connection engine
db_conn_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}"
engine = create_engine(db_conn_string)


In [181]:
# Declarative base is used to create the Object Relational Mapping of the tables in the database
Base = declarative_base()

# Define the User table
class User(Base):
    __tablename__ = "users"
    user_id = Column(Integer, primary_key=True, unique=True)
    name = Column(String)
    created_time = Column(Date)
    private = Column(Boolean)
    post_count = Column(Integer)
    verified_status = Column(Boolean)
    photos = relationship("Photo", back_populates="user")
    followers = relationship("Follow", foreign_keys='[Follow.user_id]', back_populates="user")
    following = relationship("Follow", foreign_keys='[Follow.follower_user_id]', back_populates="follower")

# Define the Photo table
class Photo(Base):
    __tablename__ = "photos"
    photo_id = Column(Integer, primary_key=True, unique=True)
    image_link = Column(String)
    user_id = Column(Integer, ForeignKey("users.user_id"), index=True)
    created_date = Column(Date)
    insta_filter_used = Column(Boolean)
    photo_type = Column(String)
    user = relationship("User", back_populates="photos")

# Define the Tag table
class Tag(Base):
    __tablename__ = "tags"
    tag_id = Column(Integer, primary_key=True, unique=True)
    tag_text = Column(String)
    created_time = Column(Date)
    location = Column(String)

# Define the Comment table
class Comment(Base):
    __tablename__ = "comments"
    comment_id = Column(Integer, primary_key=True, unique=True)
    user_id = Column(Integer, ForeignKey("users.user_id"), index=True)
    photo_id = Column(Integer, ForeignKey("photos.photo_id"), index=True)
    created_time = Column(DateTime)
    posted_date = Column(String)
    comment = Column(String)
    emoji_used = Column(Boolean)
    hashtags_used_count = Column(Integer)

# Define the Like table
class Like(Base):
    __tablename__ = "likes"
    like_id = Column(Integer, primary_key=True, unique=True)
    user_id = Column(Integer, ForeignKey("users.user_id"), index=True)
    photo_id = Column(Integer, ForeignKey("photos.photo_id"), index=True)
    created_time = Column(Date)
    following_or_not = Column(Boolean)
    like_type = Column(String)

# Define the Follow table
class Follow(Base):
    __tablename__ = "follows"
    follow_id = Column(Integer, primary_key=True, unique=True)
    follower_user_id = Column(Integer, ForeignKey("users.user_id"), index=True)
    user_id = Column(Integer, ForeignKey("users.user_id"), index=True)
    created_time = Column(Date)
    is_follower_active = Column(Integer)
    followee_acc_status = Column(String)
    user = relationship("User", foreign_keys=[user_id], back_populates="followers")
    follower = relationship("User", foreign_keys=[follower_user_id], back_populates="following")

# Define the Interactions Table which is the central fact table of the database
class Interactions(Base):
    __tablename__ = "interactions"
    interaction_id = Column(Integer, primary_key=True, unique=True)
    user_id = Column(Integer, ForeignKey("users.user_id"), index=True)
    photo_id = Column(Integer, ForeignKey("photos.photo_id"), index=True)
    tag_id = Column(Integer, ForeignKey("tags.tag_id"), index=True)
    comment_id = Column(Integer, ForeignKey("comments.comment_id"), index=True)
    like_id = Column(Integer, ForeignKey("likes.like_id"), index=True)
    follow_id = Column(Integer, ForeignKey("follows.follow_id"), index=True)
    interaction_date = Column(Date)
    interaction_type = Column(String)

# Create the database if it doesn't exist, which it shouldn't
if not database_exists(engine.url):
    create_database(engine.url)

# Using the metadata from the Base class, create the tables in the database
Base.metadata.create_all(bind=engine, checkfirst=True)

# Create a session to create the tables
Session = sessionmaker(bind=engine)
session = Session()

# Insert data into the tables 
df_users.to_sql("users", engine, if_exists='append', index=False)
df_photos.to_sql("photos", engine, if_exists='append', index=False)
df_likes.to_sql("likes", engine, if_exists='append', index=False)
df_comments.to_sql("comments", engine, if_exists='append', index=False)
df_follows.to_sql("follows", engine, if_exists='append', index=False)
df_tags.to_sql("tags", engine, if_exists='append', index=False)
df_photo_tags.to_sql("photo_Tags", engine, if_exists='append', index=False)
df_interactions.to_sql("interactions", engine, if_exists='append', index=False)

# Commit the changes and close the session
session.commit()
session.close()


### Test Query the Database

In [182]:
Session = sessionmaker(bind=engine)
session = Session()

# Query and print the # of photos in the photos table
photo_count = session.query(Photo).count()
print(f"Number of photos in the photos table: {photo_count}")

# Query and print the # of users in the users table
user_count = session.query(User).count()
print(f"Number of users in the users table: {user_count}")

# Query the count of distinct tags in the tags table
tag_count = session.query(Tag).count()
print(f"Number of tags in the tags table: {tag_count}")

# Query and print the # of likes in the interactions table
like_count = session.query(Interactions).filter(Interactions.interaction_type == 'like').count()
print(f"Number of likes in the interactions table: {like_count}")

# Query and print the # of comments in the interactions table
comment_count = session.query(Interactions).filter(Interactions.interaction_type == 'comment').count()
print(f"Number of comments in the interactions table: {comment_count}")

# Query and print the # of follows in the interactions table
follow_count = session.query(Interactions).filter(Interactions.interaction_type == 'follow').count()
print(f"Number of follows in the interactions table: {follow_count}")

session.close()

Number of photos in the photos table: 257
Number of users in the users table: 100
Number of tags in the tags table: 21
Number of likes in the interactions table: 8782
Number of comments in the interactions table: 7488
Number of follows in the interactions table: 7623
